#### TASK 3 : NLP 
### Submission To MIDAS LAB
File Name : Step 6 -trying_bidirection_LSTM 
<br>
@Author : Vansh Gupta
<br>
Objective : Finding performace of Neural Network(bidirection_LSTM).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import (
    Dense,
    Embedding,
    LSTM,
    SpatialDropout1D,
    Bidirectional,
)
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

using goole colab for faster GPU. File was uploaded on my drive and has been fetched here.

In [2]:
!wget https://www.dropbox.com/s/a68e2myvs15cfjs/data.zip?dl=0

--2021-04-07 15:24:41--  https://www.dropbox.com/s/a68e2myvs15cfjs/data.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/a68e2myvs15cfjs/data.zip [following]
--2021-04-07 15:24:41--  https://www.dropbox.com/s/raw/a68e2myvs15cfjs/data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbfa1508e0ec579322e2b9e3f6e.dl.dropboxusercontent.com/cd/0/inline/BMK7fNSY4pyOKEVCGmkFQsCjCyyOHu0tsFWG1A4SKvVV4MyFiEcqEWHQ8q4Gi7HmiNr11F7u2CVlKSzg-xDTlMcdkafPhiaCR__x4WYpCsd9d4YNcKRhs8iC_OVF66rkGtulSYSA9Umj3EK1MA-Fp-8w/file# [following]
--2021-04-07 15:24:42--  https://ucbfa1508e0ec579322e2b9e3f6e.dl.dropboxusercontent.com/cd/0/inline/BMK7fNSY4pyOKEVCGmkFQsCjCyyOHu0tsFWG1A4SKvVV4MyFiEcqEWHQ8q4Gi7HmiNr11F7u2CVlKSzg-xDTlMcdkafPhiaCR

In [3]:
!unzip data.zip?dl=0

Archive:  data.zip?dl=0
   creating: Data/
  inflating: Data/Done_with_Description.csv  
  inflating: Data/Done_with_Labels.csv  
  inflating: Data/flipkart_com-ecommerce_sample.xlsx  
  inflating: Data/Processed_data.csv  


In [4]:
df = pd.read_csv("Data/Done_with_Description.csv")

In [5]:
df.head()

,product_category_tree,description,cat_level1,cat_level2,cat_level3,cat_level4,cat_level5,cat_level6,cleaned_text
0,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Clothing,Women's Clothing,Lingerie Sleep & Swimwear,Shorts,Alisha Shorts,NaN,feature alisha solid woman cycling short cotto...
1,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,Furniture,Living Room Furniture,Sofa Beds & Futons,NaN,NaN,NaN,fabhomedecor fabric double sofa finish color l...
2,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...,Footwear,Women's Footwear,Ballerinas,NaN,NaN,NaN,feature belly sandal wedge heel casualsaw bell...
3,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Clothing,Women's Clothing,Lingerie Sleep & Swimwear,Shorts,Alisha Shorts,NaN,feature alisha solid woman cycling short cotto...
4,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...,Pet Supplies,Grooming,Skin & Coat Care,Shampoo,NaN,NaN,specification sicons purpose arnica shampoo ge...


In [6]:
## droping rows where value of cat_level1 is na
df.dropna(subset=["cat_level1"], inplace=True)
X = df["cleaned_text"].values
y = df["cat_level1"].values
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [7]:
print(X.shape, y.shape)

(19661,) (19661,)


In [8]:
print(len(onehot_encoded[0]))

28


Tokenization

In [9]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 10000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(
    num_words=MAX_NB_WORDS,
    filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
    lower=True,
)
tokenizer.fit_on_texts(df["description"].values)
word_index = tokenizer.word_index
print("Found %s unique tokens." % len(word_index))

Found 30749 unique tokens.


Converting to sequence

In [10]:
X = tokenizer.texts_to_sequences(df["description"].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print("Shape of data tensor:", X.shape)

Shape of data tensor: (19661, 250)


Converting labels to categorical

In [11]:
Y = pd.get_dummies(df["cat_level1"]).values
print("Shape of label tensor:", Y.shape)

Shape of label tensor: (19661, 28)


Test - Train Split

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.10, random_state=42
)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(17694, 250) (17694, 28)
(1967, 250) (1967, 28)


Model architecture

In [13]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(
    Bidirectional(LSTM(EMBEDDING_DIM, dropout=0.2, recurrent_dropout=0.2))
)
model.add(Dense(28, activation="softmax"))
model.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 100)          1000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 250, 100)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 28)                5628      
Total params: 1,166,428
Trainable params: 1,166,428
Non-trainable params: 0
_________________________________________________________________
None


Below Model was trained for 15 epoches . Best accuracy was given at epoch 10 with training accuracy of 93.75% and validation accuracy of 91.45%

In [17]:
epochs = 15
batch_size = 64

history = model.fit(
    X_train,
    Y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1,
    callbacks=[
        EarlyStopping(monitor="val_loss", patience=3, min_delta=0.0001)
    ],
)

Epoch 1/15
  2/249 [..............................] - ETA: 9:17 - loss: 0.2382 - accuracy: 0.9375 

KeyboardInterrupt: ignored

In [15]:
accr = model.evaluate(X_test, Y_test)
print(
    "Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}".format(accr[0], accr[1])
)

62/62 [==============================] - 11s 171ms/step - loss: 0.4065 - accuracy: 0.8927
Test set
  Loss: 0.406
  Accuracy: 0.893
